In [ ]:
#Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
#SPDX-License-Identifier: MIT-0

In [ ]:
%store -r target_dir_s3
%store -r bucket_name

In [ ]:
!pip install -q -U opensearch-py
!pip install -q boto3==1.34.144

In [ ]:
import boto3
import json
import pprint as pp
import random

In [ ]:
sts_client = boto3.client('sts')
s3_client = boto3.client('s3')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)

# Amazon Bedrock Knowledge Base (KB)

Knowledge bases for Amazon Bedrock allows you to integrate proprietary information into your generative-AI applications. Using the Retrieval Augment Generation (RAG) technique, a knowledge base searches your data to find the most useful information and then uses it to answer natural language questions. 

A knowledge base can be used not only to answer user queries, and analyze documents, but also to augment prompts provided to foundation models by providing context to the prompt. When answering user queries, the knowledge base retains conversation context. The knowledge base also grounds answers in citations so that users can find further information by looking up the exact text that a response is based on and also check that the response makes sense and is factually correct.

## Create Policies and OpenSearch serverless collection

The next code cell imports the necessary Python libraries and defines a function `create_opensearch_collection` that creates an OpenSearch Serverless Collection. This function takes two arguments: `collection_name` (the desired name for the collection) and `open_search_access_role` (the ARN of the IAM role that should have access to the collection). It performs the following steps:

1. Initializes the Boto3 client for OpenSearch Serverless.
2. Defines the network security policy and encryption security policy for the collection.
3. Creates the network security policy and encryption security policy using the OpenSearch Serverless client.
4. If an `open_search_access_role` is provided, it creates a data access policy that grants the specified role permissions to perform various operations on the collection and its indices.
5. Creates the OpenSearch Serverless Collection with the specified name and type `VECTORSEARCH`.
6. Returns the names of the created security policies and the collection response.

In [ ]:
def create_opensearch_collection(collection_name, open_search_access_role):
    # Initialize boto3 clients
    opensearch_client = boto3.client('opensearchserverless')

    # Define network security policy
    network_security_policy = json.dumps(
        [
            {
                "Rules": [
                {
                    "Resource": [
                    f"collection/{collection_name}"
                    ],
                    "ResourceType": "dashboard"
                },
                {
                    "Resource": [
                    f"collection/{collection_name}"
                    ],
                    "ResourceType": "collection"
                }
                ],
                "AllowFromPublic": True
            }
            ]
    )

    
    encryption_security_policy = json.dumps(
        {
            "Rules": [
                {
                    "Resource": [
                        f"collection/{collection_name}"
                    ],
                    "ResourceType": "collection",
                }
            ],
            "AWSOwnedKey": True
        },
        indent=2
    )

    # Create network security policy
    net_policy_response = opensearch_client.create_security_policy(
        name=f"{collection_name}-network-policy",
        policy=network_security_policy,
        type='network'
    )
    network_policy_name = net_policy_response["securityPolicyDetail"]["name"]


    # Create encryption security policy
    enc_policy_response = opensearch_client.create_security_policy(
        name=f"{collection_name}-security-policy",
        policy=encryption_security_policy,
        type='encryption'
    )
    encryption_policy_name = enc_policy_response["securityPolicyDetail"]["name"]
    
    # Create data access policy if the access role is provided
    data_access_policy_name = ""

    if open_search_access_role:
        data_access_policy = json.dumps(
            [
                {
                    "Rules": [
                    {
                        "Resource": [
                        f"collection/{collection_name}"
                        ],
                        "Permission": [
                        "aoss:CreateCollectionItems",
                        "aoss:DeleteCollectionItems",
                        "aoss:UpdateCollectionItems",
                        "aoss:DescribeCollectionItems"
                        ],
                        "ResourceType": "collection"
                    },
                    {
                        "Resource": [
                        f"index/{collection_name}/*"
                        ],
                        "Permission": [
                        "aoss:CreateIndex",
                        "aoss:DeleteIndex",
                        "aoss:UpdateIndex",
                        "aoss:DescribeIndex",
                        "aoss:ReadDocument",
                        "aoss:WriteDocument"
                        ],
                        "ResourceType": "index"
                    }
                    ],
                    "Principal": [open_search_access_role],
                    "Description": "data-access-rule"
                }
            ]
        )


        data_access_policy_name = f"{collection_name}-access"
        if len(data_access_policy_name) > 32:
            raise ValueError('Policy name exceeds maximum length of 32 characters')

        cfn_access_policy_response = opensearch_client.create_access_policy(
            name=data_access_policy_name,
            description='Policy for data access',
            policy=data_access_policy,
            type='data',
        )


    # Create OpenSearch collection
    collection_response = opensearch_client.create_collection(
        name=collection_name,
        type='VECTORSEARCH'
    )

    return encryption_policy_name, network_policy_name, data_access_policy_name, collection_response



In this code cell, we first retrieve identity of logged in user/role and then invoke create_opensearch_collection function created in the previous cell to create open search collection.
The function returns the names of the created security policies and the collection response, which is printed at the end of the cell.

In [ ]:
# Get the caller identity ARN
sts_client = boto3.client('sts')
caller_identity = sts_client.get_caller_identity()
identity_arn = caller_identity['Arn']

#create the collection
collection_name = 'faq-collection'
encryption_policy_name, network_policy_name, data_access_policy_name, collection_response = create_opensearch_collection(collection_name, identity_arn)

print(collection_response)

This code cell extracts the collection ID and region from the collection response obtained in the previous cell. It then constructs the OpenSearch Serverless endpoint URL (`os_host`) using the collection ID, region, and the domain suffix `.aoss.amazonaws.com`. Finally, it prints the `os_host` value, which can be used to connect to the OpenSearch Serverless cluster and perform various operations.

In [ ]:
collection_id = collection_response['createCollectionDetail']['id']

collection_arn = collection_response['createCollectionDetail']['arn']

region = collection_response['createCollectionDetail']['arn'].split(":")[3]

os_host = ".".join([collection_id, region, "aoss.amazonaws.com"])

print(os_host)

## Create vector Index

### Bedrock execution role (and associated policies) for KB creation

we need to create a specific role for the KB creation and attach associated policies to allow access to foundation models and S3 bucket

In [ ]:
iam_client = boto3_session.client('iam')

kb_suffix = "FAQ_WS_" + str(random.randint(0,1000))

account_number = boto3.client('sts').get_caller_identity().get('Account')

bedrock_execution_role_name = f'AmazonBedrockExecutionRoleForKnowledgeBase_{kb_suffix}'
s3_policy_name = f'AmazonBedrockS3PolicyForKnowledgeBase_{kb_suffix}'
fm_policy_name = f'AmazonBedrockFoundationModelPolicyForKnowledgeBase_{kb_suffix}'
AOSSAccessPolicy_name = f'AmazonBedrockAossPolicyForKnowledgeBase_{kb_suffix}'

embedding_model_id = "cohere.embed-english-v3"

def create_bedrock_execution_role(bucket_name):
    foundation_model_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:InvokeModel",
                ],
                "Resource": [
                    f"arn:aws:bedrock:{region_name}::foundation-model/{embedding_model_id}"
                ]
            }
        ]
    }

    s3_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "s3:GetObject",
                    "s3:ListBucket"
                ],
                "Resource": [
                    f"arn:aws:s3:::{bucket_name}",
                    f"arn:aws:s3:::{bucket_name}/*"
                ],
                "Condition": {
                    "StringEquals": {
                        "aws:ResourceAccount": f"{account_number}"
                    }
                }
            }
        ]
    }

    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    # create policies based on the policy documents
    fm_policy = iam_client.create_policy(
        PolicyName=fm_policy_name,
        PolicyDocument=json.dumps(foundation_model_policy_document),
        Description='Policy for accessing foundation model',
    )

    s3_policy = iam_client.create_policy(
        PolicyName=s3_policy_name,
        PolicyDocument=json.dumps(s3_policy_document),
        Description='Policy for reading documents from s3')
    

    # Define the policy document
    policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "aoss:*",
                "Resource": "*"
            }
        ]
    }

    # Create the IAM policy to access aoss
    aossAccessPolicy = iam_client.create_policy(
        PolicyName=AOSSAccessPolicy_name,
        PolicyDocument=json.dumps(policy_document)
    )


    # create bedrock execution role
    bedrock_kb_execution_role = iam_client.create_role(
        RoleName=bedrock_execution_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_document),
        Description='Amazon Bedrock Knowledge Base Execution Role for accessing OSS and S3',
        MaxSessionDuration=3600
    )

    # fetch arn of the policies and role created above
    bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']
    s3_policy_arn = s3_policy["Policy"]["Arn"]
    fm_policy_arn = fm_policy["Policy"]["Arn"]
    aossAccessPolicy_arn = aossAccessPolicy["Policy"]["Arn"]

    # attach policies to Amazon Bedrock execution role
    iam_client.attach_role_policy(
        RoleName=bedrock_kb_execution_role["Role"]["RoleName"],
        PolicyArn=fm_policy_arn
    )
    iam_client.attach_role_policy(
        RoleName=bedrock_kb_execution_role["Role"]["RoleName"],
        PolicyArn=s3_policy_arn
    )
    iam_client.attach_role_policy(
        RoleName=bedrock_kb_execution_role["Role"]["RoleName"],
        PolicyArn=aossAccessPolicy_arn
    )
    sm_policy_arn = 'arn:aws:iam::aws:policy/SecretsManagerReadWrite'
    iam_client.attach_role_policy(
        RoleName=bedrock_kb_execution_role["Role"]["RoleName"],
        PolicyArn=sm_policy_arn
    )
    return bedrock_kb_execution_role

In [ ]:
kb_role_response = create_bedrock_execution_role(bucket_name)
kb_role_arn = kb_role_response['Role']['Arn']

In [ ]:
kb_role_name = kb_role_response['Role']["RoleName"]
kb_role_name

### Adding the newly created KB role to the data access policy for our openSearch collection

In [ ]:
# Create an OpenSearch Serverless client
opss_client = boto3.client('opensearchserverless')

# Retrieve the existing data access policy
try:
    response = opss_client.get_access_policy(
        name=data_access_policy_name,
        type='data'
    )
    existing_policy = response['accessPolicyDetail']['policy']
    policy_version = response['accessPolicyDetail']['policyVersion']
except opss_client.exceptions.ResourceNotFoundException:
    print(f"Data access policy for collection '{collection_name}' not found.")
    existing_policy = []
    policy_version = None

# Add the IAM role ARN as a principal in the first rule
if existing_policy:
    existing_policy[0]['Principal'].append(kb_role_arn)
else:
    existing_policy = [{
        'Principal': [kb_role_arn],
        'Rules': [],
        'Description': 'Data access policy'
    }]

# Update the data access policy
try:
    response = opss_client.update_access_policy(
        name=data_access_policy_name,
        type='data',
        policy=json.dumps(existing_policy),
        policyVersion=policy_version
    )
    print(f"Successfully updated data access policy for collection '{collection_name}'.")
except Exception as e:
    print(f"Error updating data access policy: {e}")

## Create semantic search engine with Amazon OpenSearch Service Serverless

### Opensearch data access policy update

To access our opensearch serverless collection, we need to update its data policy update with the current user

The code cell sets up an AWS OpenSearch Serverless client by assigning the service name, obtaining AWS credentials, creating an authentication object, and then using the connect_to_aoss function to create the client with the authentication object and the OpenSearch host.

In [ ]:
from opensearchpy import (
    OpenSearch,
    RequestsHttpConnection,
    AWSV4SignerAuth
)

#connect to opensearch serverless
#auth : AWSV4SignerAuth
#host: <opensearchid>.us-east-1.aoss.amazonaws.com
def connect_to_aoss(auth, host):
    try:
        # create an opensearch client and use the request-signer
        aoss_client = OpenSearch(
            hosts=[{'host': host, 'port': 443}],
            http_auth=auth,
            use_ssl=True,
            verify_certs=True,
            connection_class=RequestsHttpConnection,
            pool_maxsize=20,
        )
        return aoss_client
    except Exception as e:
        print(e)
        return None 
    
#opensearch serverless service, aka aoss
service = 'aoss'

#get an Auth object to call aoss
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region_name, service)

#LLMUtils.connect_to_aoss() can be found in lib/src/utils/ folder.
aoss_client = connect_to_aoss(auth, os_host)


### Create OpenSearch index

In the next few cells, we define the index name, data columns to be included, and the index configuration. We then create the index if it doesn't already exist


In the following code cell, an OpenSearch index named "movies-index" is defined. The code also specifies a list of data columns that will be added to this index, including information about movies such as their TMDB ID, original language, title, description, genres, release year, keywords, director, actors, popularity score, popularity score bins, average vote rating, and average vote rating bins.

In [ ]:
#index configuration. note that we're adding both text metadata as well as the vector_index property that will be storing our embedding for each title.
# For additional information on the K-NN index configuration, please read the below documentation.
#https://opensearch.org/docs/latest/field-types/supported-field-types/knn-vector/
#https://opensearch.org/docs/latest/search-plugins/knn/knn-index/

#opensearch index name
index_name = "faq-vector-index"

index_body = {
  "settings": {
    "index": {
      'number_of_shards': 1,
      "number_of_replicas": 0,
      "knn": True,
      "knn.algo_param.ef_search": 512
    }
  },
  "mappings": {
    "properties": {
      "text": {"type": "text"},
      "text-metadata": {"type": "text"},
      "vector_index": {
        "type": "knn_vector",
        "dimension": 1024, #if you use cohere: dimension of the embedding is 1024, for titan: 1536
        "method": {
          "name": "hnsw",
          "space_type": "l2",
          "engine": "faiss",
          "parameters": {
            "ef_construction": 512,
            "m": 16
          }
        }
      }
    }
  }
}

This section of the code checks if the specified index already exists in OpenSearch. If the index does not exist, it creates a new index with the provided index body. 

In [ ]:
#get a list of the indexes already existing
indexes = aoss_client.indices.get_alias("*")
indexes_list = list(indexes.keys())

#check if index doesn't already exist and create it
if index_name not in indexes_list:
    print('Creating index:\n')
    create_response = aoss_client.indices.create(index_name, body=index_body)
    print(create_response)
else:
    print("index already exists")

Here, we display information about the newly created index

The code cell retrieves and displays details about a previously created OpenSearch index, including its mapping (fields and data types), settings (configuration options like shards and replicas), and any associated aliases

In [ ]:
#display information on the index you just created

# Get index mapping
response = aoss_client.indices.get_mapping(index=index_name)
pp.pprint(response) 

# Get index settings
response = aoss_client.indices.get_settings(index=index_name)
pp.pprint(response)

# Get index aliases
response = aoss_client.indices.get_alias(index=index_name) 
pp.pprint(response)

## Create Amazon Bedrock Knowledge Base

Code reuse from following git repo:
https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/02_KnowledgeBases_and_RAG/0_create_ingest_documents_test_kb.ipynb

### Create the actual KB finally

In [ ]:
opensearchServerlessConfiguration = {
            "collectionArn": collection_arn,
            "vectorIndexName": index_name,
            "fieldMapping": {
                "vectorField": "vector_index",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

# Ingest strategy - How to ingest data from the data source
chunkingStrategyConfiguration = {
    "chunkingStrategy": "FIXED_SIZE",
    "fixedSizeChunkingConfiguration": {
        "maxTokens": 512,
        "overlapPercentage": 20
    }
}

# The data source to ingest documents from, into the OpenSearch serverless knowledge base index
s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket_name}",
    # "inclusionPrefixes":["*.*"] # you can use this if you want to create a KB using data within s3 prefixes.
}

# The embedding model used by Bedrock to embed ingested documents, and realtime prompts
embeddingModelArn = f"arn:aws:bedrock:{region_name}::foundation-model/{embedding_model_id}"

kb_name = f"evaluation-ws-knowledge-base-{kb_suffix}"
kb_description = "FAQ KB"

In [ ]:
create_kb_response = bedrock_agent_client.create_knowledge_base(
    name = kb_name,
    description = kb_description,
    roleArn = kb_role_arn,
    knowledgeBaseConfiguration = {
        "type": "VECTOR",
        "vectorKnowledgeBaseConfiguration": {
            "embeddingModelArn": embeddingModelArn
        }
    },
    storageConfiguration = {
        "type": "OPENSEARCH_SERVERLESS",
        "opensearchServerlessConfiguration":opensearchServerlessConfiguration
    }
)

In [ ]:
kb_id = create_kb_response["knowledgeBase"]["knowledgeBaseId"]

In [ ]:
# Check KnowledgeBase 
bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb_id)

In [ ]:
# Create a DataSource in KnowledgeBase 
create_ds_response = bedrock_agent_client.create_data_source(
    name = kb_name,
    description = kb_description,
    knowledgeBaseId = kb_id,
    dataSourceConfiguration = {
        "type": "S3",
        "s3Configuration":s3Configuration
    },
    vectorIngestionConfiguration = {
        "chunkingConfiguration": chunkingStrategyConfiguration
    }
)
kb_ds = create_ds_response["dataSource"]

kb_ds_id = kb_ds["dataSourceId"]

pp.pprint(kb_ds)

In [ ]:
# Get DataSource 
bedrock_agent_client.get_data_source(knowledgeBaseId = kb_id, dataSourceId = kb_ds_id)

## Ingestion Job
During the ingestion job, Bedrock KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the AOSS vector database

In [ ]:
# Start an ingestion job
start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb_id, dataSourceId = kb_ds_id)

In [ ]:
job = start_job_response["ingestionJob"]
pp.pprint(job)

In [ ]:
import time

# Get job 
while(job['status']!='COMPLETE' ):
    get_job_response = bedrock_agent_client.get_ingestion_job(
      knowledgeBaseId = kb_id,
        dataSourceId = kb_ds_id,
        ingestionJobId = job["ingestionJobId"]
  )
    job = get_job_response["ingestionJob"]
    
    time.sleep(30)

pp.pprint(job)

Retrieve API Test Call

In [ ]:
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region_name)

query = "I cannot login to my account"

# retrieve api for fetching only the relevant context.
relevant_documents = bedrock_agent_runtime_client.retrieve(
    retrievalQuery= {
        'text': query
    },
    knowledgeBaseId=kb_id,
    retrievalConfiguration= {
        'vectorSearchConfiguration': {
            'numberOfResults': 3 # will fetch top 3 documents which matches closely with the query.
        }
    }
)

In [ ]:
relevant_documents['retrievalResults'][0]

In [ ]:
for document in relevant_documents['retrievalResults']:
    document['content']['text']
    print(f"text:{document['content']['text']}")
    print(f"source:{document['location']['s3Location']['uri'].split('/')[-1]}")
    print(f"score:{document['score']}")
    print('-----------------')

Retrieve and Generate KB API Test Call. Feel free to use different models to test the output.

In [ ]:
session = boto3.Session()
bedrock = session.client(service_name='bedrock')
list_FMs = bedrock.list_foundation_models(byProvider='Anthropic')
list_FMs["modelSummaries"]

In [ ]:
model_arn = "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"
#model_arn = "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-haiku-20240307-v1:0"

retrieve_and_generate_response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        'text': query
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn
        }
    },
)

In [ ]:
print(retrieve_and_generate_response['citations'][0]['generatedResponsePart']['textResponsePart']['text'])

In [ ]:
%store kb_id
%store collection_id
%store collection_name
%store encryption_policy_name
%store network_policy_name
%store data_access_policy_name
%store os_host
%store index_name
%store identity_arn
%store kb_role_arn
%store kb_role_name